# Introducing PySpark SQL

While we can directly analyze data using Spark’s Resilient Distributed Datasets (RDDs), we may not always want to perform complicated analysis directly on RDDs. Luckily, Spark offers a module called Spark SQL that can make common data analysis tasks simpler and faster. In this lesson, we’ll introduce Spark SQL and demonstrate how it can be a powerful tool for accelerating the analysis of distributed datasets.

The name Spark SQL is an umbrella term, as there are several ways to interact with data when using this module. We’ll cover two of these methods using the PySpark API:

* First, we’ll learn the basics of inspecting and querying data in a Spark DataFrame.

* Then, we’ll perform these same operations using standard SQL directly in our PySpark code.

Before using either method, we must start a SparkSession, the entry point to Spark SQL. The session is a wrapper around a sparkContext and contains all the metadata required to start working with distributed data.

The code below uses SparkSession.builder to set configuration parameters and create a new session. In the following example, we set one configuration parameter (spark.app.name) and call the .getOrCreate() method to initialize the new SparkSession.

In [1]:
from pyspark.sql import SparkSession
import findspark
findspark.init()

spark = SparkSession.builder\
    .config('spark.app.name', 'learning_spark_sql')\
    .getOrCreate()

We can access the SparkContext for a session with SparkSession.sparkContext.

In [2]:
print(spark.sparkContext) 
# <SparkContext master=local[*] appName=learning_spark_sql>

<SparkContext master=local[*] appName=learning_spark_sql>


From here, we can use the SparkSession to create DataFrames, read external files, register tables, and run SQL queries over saved data. When we’re done with our analysis, we can clear the Spark cache and terminate the session with SparkSession.stop(). Now that we’re familiar with the basics of SparkSession, the next step is to begin using Spark SQL to interact with data!

### Practice

https://spark.apache.org/docs/latest/configuration.html#available-properties

## Creating Spark DataFrames

A PySpark SQL DataFrame is a distributed collection of data with a specific row and column structure. Under the hood, DataFrames are built on top of RDDs. Like pandas, PySpark SQL DataFrames allow a developer to analyze data more easily than by writing functions directly on underlying data.

DataFrames can be created manually from RDDs using rdd.toDF(["names", "of", "columns"]). In the example below, we create a DataFrame from a manually constructed RDD and name its columns article_title and view_count.

In [3]:
# Create an RDD from a list
hrly_views_rdd  = spark.sparkContext.parallelize([
    ["Betty_White" , 288886],
    ["Main_Page", 139564],
    ["New_Year's_Day", 7892],
    ["ABBA", 8154]
])

# Convert RDD to DataFrame
hrly_views_df = hrly_views_rdd\
    .toDF(["article_title", "view_count"])

Let’s take a look at our new DataFrame. We can use the DataFrame.show(n_rows) method to print the first n_rows of a Spark DataFrame. It can also be helpful to pass truncate=False to ensure all columns are visible.

In [4]:
hrly_views_df.show(4, truncate=False)

+--------------+----------+
|article_title |view_count|
+--------------+----------+
|Betty_White   |288886    |
|Main_Page     |139564    |
|New_Year's_Day|7892      |
|ABBA          |8154      |
+--------------+----------+



Great! Now that this data is loaded in as a DataFrame, we can access the underlying RDD with DataFrame.rdd. You likely won’t need the underlying data often, but it can be helpful to keep in mind that a DataFrame is a structure built on top of an RDD. When we check the type of hrly_views_df_rdd, we can see that it’s an RDD!

In [5]:
# Access DataFrame's underlying RDD
hrly_views_df_rdd = hrly_views_df.rdd

# Check object type
print(type(hrly_views_df_rdd)) 
# <class 'pyspark.rdd.RDD'>

<class 'pyspark.rdd.RDD'>


### Practice

1.
Because we learned about SparkSession in the first exercise, all remaining exercises in this lesson will include the code to create a SparkSession named spark for you to use. Be sure to run these cells!

Using the RDD sample_page_views, create a DataFrame named sample_page_views_df with columns named language_code, title, date, and count.

In the same code cell, add code to show the first five rows of the DataFrame. Set truncate=False to ensure all columns are visible.

In [6]:
from pyspark.sql import SparkSession
import findspark
findspark.init()

# Create a new SparkSession
spark = SparkSession\
    .builder\
    .config('spark.app.name', 'learning_spark_sql')\
    .getOrCreate()

sample_page_views  = spark.sparkContext.parallelize([
    ["en", "Statue_of_Liberty", "2022-01-01", 263],
    ["en", "Replicas_of_the_Statue_of_Liberty", "2022-01-01", 11],
    ["en", "Statue_of_Lucille_Ball" ,"2022-01-01", 6],
    ["en", "Statue_of_Liberty_National_Monument", "2022-01-01", 4],
    ["en", "Statue_of_Liberty_play"  ,"2022-01-01", 3],  
])

In [7]:
## YOUR SOLUTION HERE ##
sample_page_views_df = sample_page_views.toDF(['language_code', 'title', 'date', 'count'])

# show first 5 rows
sample_page_views_df.show(5, truncate=False)

+-------------+-----------------------------------+----------+-----+
|language_code|title                              |date      |count|
+-------------+-----------------------------------+----------+-----+
|en           |Statue_of_Liberty                  |2022-01-01|263  |
|en           |Replicas_of_the_Statue_of_Liberty  |2022-01-01|11   |
|en           |Statue_of_Lucille_Ball             |2022-01-01|6    |
|en           |Statue_of_Liberty_National_Monument|2022-01-01|4    |
|en           |Statue_of_Liberty_play             |2022-01-01|3    |
+-------------+-----------------------------------+----------+-----+



2.
Access the RDD underlying sample_page_views_df and save it as sample_page_views_rdd_restored. In the same code cell, run sample_page_views_rdd_restored.collect() to view the restored RDD.

Note: You may notice that the restored RDD is not identical to the original RDD! Although the data is the same, when we converted the data to a DataFrame, PySpark automatically wrapped the original content into a Row. Behind the scenes, rows allow for more efficient calculations over large distributed data.

In [8]:
## YOUR SOLUTION HERE ##
sample_page_views_rdd_restored = sample_page_views_df.rdd

# show restored RDD
sample_page_views_rdd_restored.collect()

[Row(language_code='en', title='Statue_of_Liberty', date='2022-01-01', count=263),
 Row(language_code='en', title='Replicas_of_the_Statue_of_Liberty', date='2022-01-01', count=11),
 Row(language_code='en', title='Statue_of_Lucille_Ball', date='2022-01-01', count=6),
 Row(language_code='en', title='Statue_of_Liberty_National_Monument', date='2022-01-01', count=4),
 Row(language_code='en', title='Statue_of_Liberty_play', date='2022-01-01', count=3)]

## Spark DataFrames from External Sources

In this exercise, we’ll learn how to pull in larger datasets from external sources. To start, we’ll be using a dataset from Wikipedia that counts views of all articles by hour. For demonstration’s sake, we’ll use the first hour of 2022. Let’s take a look at the code we might use to read a CSV of this data from a location on disk.

In [9]:
print(type(spark.read)) 
# <class 'pyspark.sql.readwriter.DataFrameReader'>

# Read CSV to DataFrame
hrly_views_df = spark.read.option('header', True).option('delimiter', ' ').option('inferSchema', True).csv('views_2022_01_01_000000.csv')

<class 'pyspark.sql.readwriter.DataFrameReader'>


There are a few things going on in this code, let’s go through them one at a time:

This code uses the SparkSession.read function to create a new DataFrameReader

The DataFrameReader has an .option('option_name', 'option_value') method that can be used to instruct Spark how exactly to read a file. In this case, we used the following options:

* .option('header', True) — Indicate the file already contains a header row. By default, Spark assumes there is no header.

* .option('delimiter', ' ') — Indicates each column is separated by a space (‘ ‘). By default, Spark assumes CSV columns are separated by commas.

* .option('inferSchema', True) — Instructs Spark to sample a subset of rows before determining each column’s type. By default, Spark will treat all CSV columns as strings.

The DataFrameReader also has a .csv('path') method which loads a CSV file and returns the result as a DataFrame. There are a few quick ways of checking that our data has been read in properly. The most direct way is checking DataFrame.show().

In [10]:
# Display first 5 rows of DataFrame
hrly_views_df.show(5, truncate=False)

+------------------+-------------------+-----------------+-------------------+
|domain            |uniq_human_visitors|uniq_bot_visitors|total_visitor_count|
+------------------+-------------------+-----------------+-------------------+
|en.m.wikipedia.org|33261399           |8400247          |41661646           |
|en.wikipedia.org  |17009339           |4851741          |21861080           |
|es.m.wikipedia.org|5668575            |1977289          |7645864            |
|ru.m.wikipedia.org|5816762            |1367179          |7183941            |
|ja.m.wikipedia.org|5396108            |1325212          |6721320            |
+------------------+-------------------+-----------------+-------------------+
only showing top 5 rows



Looks Good! In this exercise, we used a DataFrameReader to pull a CSV from disk into our local Spark environment. However, Spark can read a wide variety of file formats. You can refer to the PySpark documentation to explore all available DataFrameReader options and file formats. In the following exercise, we’ll start to analyze the contents of this file.

https://spark.apache.org/docs/latest/api/python/search.html?q=DataFrameReader

### Practice


The file wiki_uniq_march_2022.csv contains the estimated count of unique visitors to each Wikipedia domain on March 1st, 2022. The file has the following layout:

* Site/Project Name (string)

* Estimated Human Visitors (int)

* Estimated Bot Visitors (int)

* Total Traffic (int)

You can read more about how Wikipedia estimates these values here.

First, let’s load the data from wiki_uniq_march_2022.csv as a DataFrame named wiki_uniq_df and display the first 10 rows in the notebook with truncate = False. For the moment, do not add any options when reading the file.

In [11]:
from pyspark.sql import SparkSession
import findspark
findspark.init()

# Create a new SparkSession
spark = SparkSession\
    .builder\
    .config('spark.app.name', 'learning_spark_sql')\
    .getOrCreate()

In [12]:
## YOUR SOLUTION HERE ##
wiki_uniq_df = spark.read.csv('wiki_uniq_march_2022.csv')

# show the first 10 rows
wiki_uniq_df.show(10, truncate=False)

+------------------+-------------------+-----------------+-------------------+
|_c0               |_c1                |_c2              |_c3                |
+------------------+-------------------+-----------------+-------------------+
|domain            |uniq_human_visitors|uniq_bot_visitors|total_visitor_count|
|en.m.wikipedia.org|33261399           |8400247          |41661646           |
|en.wikipedia.org  |17009339           |4851741          |21861080           |
|es.m.wikipedia.org|5668575            |1977289          |7645864            |
|ru.m.wikipedia.org|5816762            |1367179          |7183941            |
|ja.m.wikipedia.org|5396108            |1325212          |6721320            |
|de.m.wikipedia.org|4439596            |853251           |5292847            |
|fr.m.wikipedia.org|3798528            |904567           |4703095            |
|ru.wikipedia.org  |2852296            |687501           |3539797            |
|es.wikipedia.org  |2460489            |962516      

We’ve read the file, but the result doesn’t quite look right! This file has a header row.

Pass the option to the DataFrameReader that will read the file and create a header from the first row. Name this DataFrame wiki_uniq_w_header_df. In the same code cell, show the first 10 rows of wiki_uniq_w_header_df with truncate=False to check the header has loaded properly.

In [13]:
## YOUR SOLUTION HERE ##
wiki_uniq_w_header_df = spark.read\
    .option('header', True)\
    .csv('wiki_uniq_march_2022.csv')

# show the first 10 rows
wiki_uniq_w_header_df.show(10, truncate=False)

+------------------+-------------------+-----------------+-------------------+
|domain            |uniq_human_visitors|uniq_bot_visitors|total_visitor_count|
+------------------+-------------------+-----------------+-------------------+
|en.m.wikipedia.org|33261399           |8400247          |41661646           |
|en.wikipedia.org  |17009339           |4851741          |21861080           |
|es.m.wikipedia.org|5668575            |1977289          |7645864            |
|ru.m.wikipedia.org|5816762            |1367179          |7183941            |
|ja.m.wikipedia.org|5396108            |1325212          |6721320            |
|de.m.wikipedia.org|4439596            |853251           |5292847            |
|fr.m.wikipedia.org|3798528            |904567           |4703095            |
|ru.wikipedia.org  |2852296            |687501           |3539797            |
|es.wikipedia.org  |2460489            |962516           |3423005            |
|it.m.wikipedia.org|2806943            |566876      

In [14]:
# show the data types
wiki_uniq_w_header_df.dtypes

[('domain', 'string'),
 ('uniq_human_visitors', 'string'),
 ('uniq_bot_visitors', 'string'),
 ('total_visitor_count', 'string')]

This result is better, but we haven’t specified the types for the DataFrame yet. Check the data types for each column in wiki_uniq_w_header_df by running the provided code.

It looks like PySpark defaults to strings as the data types for all columns when we don’t specify them somehow.

Read the data in again, this time passing an option to the DataFrameReader that will tell Spark to sample rows to determine the file schema. Name this DataFrame wiki_uniq_w_schema_df. In the same code cell, run the provided code to check the data types in the new DataFrame.

In [15]:
## YOUR SOLUTION HERE ##
wiki_uniq_w_schema_df = spark.read\
    .option('header', True)\
    .option('delimiter', ',')\
    .option('inferSchema', True)\
    .csv('wiki_uniq_march_2022.csv')

# show the data types
wiki_uniq_w_schema_df.dtypes

[('domain', 'string'),
 ('uniq_human_visitors', 'int'),
 ('uniq_bot_visitors', 'int'),
 ('total_visitor_count', 'int')]

## Inspecting and Cleaning Data With PySpark

In this exercise, we’re going to start to analyze our pageview data and learn how Spark can help with data exploration. Like Pandas, Spark DataFrames offer a series of operations for cleaning, inspecting, and transforming data. Earlier in the lesson, we mentioned that all DataFrames have a schema that defines their structure, columns, and datatypes. We can use DataFrame.printSchema() to show a DataFrame’s schema.

In [16]:
# Display DataFrame schema
hrly_views_df.printSchema()

root
 |-- domain: string (nullable = true)
 |-- uniq_human_visitors: integer (nullable = true)
 |-- uniq_bot_visitors: integer (nullable = true)
 |-- total_visitor_count: integer (nullable = true)



We can then use DataFrame.describe() to see a high-level summary of the data by column. The result of DataFrame.describe() is a DataFrame in itself, so we append .show() to get it to display in our notebook.

In [17]:
hrly_views_df_desc = hrly_views_df.describe()
hrly_views_df_desc.show(truncate=False)

+-------+----------------+-------------------+-----------------+-------------------+
|summary|domain          |uniq_human_visitors|uniq_bot_visitors|total_visitor_count|
+-------+----------------+-------------------+-----------------+-------------------+
|count  |760             |760                |760              |760                |
|mean   |NULL            |155413.0394736842  |51431.0552631579 |206844.09473684212 |
|stddev |NULL            |1435327.5409314982 |376318.441663093 |1809320.9789242456 |
|min    |aa.wikibooks.org|0                  |170              |1005               |
|max    |zu.wikipedia.org|33261399           |8400247          |41661646           |
+-------+----------------+-------------------+-----------------+-------------------+



Because this data was taken from the first hour of the month, it looks like the column monthly_count only contains zeros. Because it contains no meaningful information, we can drop this field with DataFrame.drop("columns", "to", "drop").

The data is starting to look pretty good, but let’s make one more adjustment. The column article_title is a bit misleading: it seems this data contains articles, files, image pages, and wikipedia metadata pages. We can replace this misleading header with a better name using DataFrame.withColumnRenamed().

You may have noticed that Spark assigned all columns nullable = true. Intuitively, we know that article_title shouldn’t be null, but when the DataFrameReader reads a CSV, it assigns nullable = true to all columns. This is fine for now, but in some scenarios, you may wish to explicitly define a file’s schema. If interested, you can refer to PySpark’s documentation on defining a file’s schema. https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.DataFrame.schema.html

### Practice

The code to read in the Wikipedia unique data has already been written. Let’s find out if the number and types of columns in the DataFrame look correct. Print the schema of the DataFrame.

In [18]:
from pyspark.sql import SparkSession
import findspark
findspark.init()

spark = SparkSession \
    .builder \
    .appName("learning_spark_sql") \
    .getOrCreate()

# read in the Wikipedia unique visitors dataset
uniq_views_df = spark.read\
    .option('header', True) \
    .option('delimiter', ',') \
    .option('inferSchema', True) \
    .csv("wiki_uniq_march_2022.csv")

In [19]:
## YOUR SOLUTION HERE ##
uniq_views_df.printSchema()

root
 |-- domain: string (nullable = true)
 |-- uniq_human_visitors: integer (nullable = true)
 |-- uniq_bot_visitors: integer (nullable = true)
 |-- total_visitor_count: integer (nullable = true)



Let’s summarize this data and find out the mean total number of visitors per site. Save a high-level summary of the DataFrame to a new DataFrame named uniq_counts_df_desc and display it in the notebook.

In [20]:
## YOUR SOLUTION HERE ##
uniq_views_df_desc = uniq_views_df.describe()

# show summary
uniq_views_df_desc.show()

+-------+----------------+-------------------+-----------------+-------------------+
|summary|          domain|uniq_human_visitors|uniq_bot_visitors|total_visitor_count|
+-------+----------------+-------------------+-----------------+-------------------+
|  count|             760|                760|              760|                760|
|   mean|            NULL|  155413.0394736842| 51431.0552631579| 206844.09473684212|
| stddev|            NULL| 1435327.5409314982| 376318.441663093| 1809320.9789242456|
|    min|aa.wikibooks.org|                  0|              170|               1005|
|    max|zu.wikipedia.org|           33261399|          8400247|           41661646|
+-------+----------------+-------------------+-----------------+-------------------+



We found out that the mean total number of visitors per site is about 206,844. Let’s assume our analysis is focused on only the uniq_human_visitors. Write code to drop total_visitor_count and uniq_bot_visitors. Save the result to a DataFrame named uniq_counts_human_df and show the first 5 rows.

In [21]:
## YOUR SOLUTION HERE ##
uniq_counts_human_df = uniq_views_df.drop('uniq_bot_visitors','total_visitor_count')

# show the first 5 rows
uniq_counts_human_df.show(5)

+------------------+-------------------+
|            domain|uniq_human_visitors|
+------------------+-------------------+
|en.m.wikipedia.org|           33261399|
|  en.wikipedia.org|           17009339|
|es.m.wikipedia.org|            5668575|
|ru.m.wikipedia.org|            5816762|
|ja.m.wikipedia.org|            5396108|
+------------------+-------------------+
only showing top 5 rows



Finally, let’s rename the column uniq_human_visitors to something a bit more descriptive. Rename uniq_human_visitors to unique_site_visitors. Save the new DataFrame as uniq_counts_final_df and show the first 5 rows.

In [22]:
## YOUR SOLUTION HERE ##
uniq_counts_final_df = uniq_counts_human_df.withColumnRenamed('uniq_human_visitors', 'unique_site_visitors' )

# show the first 5 rows
uniq_counts_final_df.show(5)

+------------------+--------------------+
|            domain|unique_site_visitors|
+------------------+--------------------+
|en.m.wikipedia.org|            33261399|
|  en.wikipedia.org|            17009339|
|es.m.wikipedia.org|             5668575|
|ru.m.wikipedia.org|             5816762|
|ja.m.wikipedia.org|             5396108|
+------------------+--------------------+
only showing top 5 rows

